# Code Business Analytics Assignment 3 - group ...

--------------------------------------------------

# Importing Data

In [22]:
!pip install google.cloud

In [ ]:
import pandas as pd
import json
import numpy as np
from sklearn.cluster import KMeans
from sklearn import ensemble
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn import metrics
from pprint import pprint

import _pickle as cPickle
# from google.cloud import storage

In [3]:
data_path = 'Data/DataTrain.txt'
feature_path = 'Features/'
model_path = 'model/'
tmp_dir = 'tmp/'

In [11]:
#Importing the data from .txt file
data = pd.read_csv(data_path, sep=" ", header=None)
data.columns = ['engine_id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 'NA', 'NA']
del data['NA']

# Creating and adding the RUL to the dataframe
RUL_list = []
for engine in set(data['engine_id']):
    max_cycle = data.loc[data['engine_id'] == engine].cycle.max()
    
    for cycle in list(data.loc[data['engine_id'] == engine].cycle):
        RUL_list.append(max_cycle - cycle +1)    
        

data.insert(2, 'RUL', RUL_list)

data.head()
dataset = data
dataset.head()

,engine_id,cycle,RUL,setting1,setting2,setting3,s1,s2,s3,s4,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,192,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,191,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,190,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,189,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,188,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [24]:
engine = dataset.iloc[:,0].to_list()
cycle = dataset.iloc[:,1].to_list()

# Clustering the data
X_cluster = data[['setting1', 'setting2', 'setting3']]

# creates the clusters
kmeans = KMeans(n_clusters=3).fit(X_cluster)
data['settings_clusters'] = kmeans.predict(X_cluster)

features = data.columns[3:-1]
for feature in features:
    # Creating min, max and delta variables
    data['max_' + feature] = data.groupby('engine_id')[feature].cummax()
    data['min_' + feature] = data.groupby('engine_id')[feature].cummin()

    data['delta_' + feature] = data.groupby('engine_id')[feature].diff()
    data['delta_' + feature].fillna(0, inplace=True)

data.to_parquet(feature_path + 'RUL_features.parquet', compression='GZIP')

In [13]:
[print(column) for column in data.columns];

engine_id
cycle
RUL
setting1
setting2
setting3
s1
s2
s3
s4
s5
s6
s7
s8
s9
s10
s11
s12
s13
s14
s15
s16
s17
s18
s19
s20
s21
settings_clusters
max_setting1
min_setting1
delta_setting1
max_setting2
min_setting2
delta_setting2
max_setting3
min_setting3
delta_setting3
max_s1
min_s1
delta_s1
max_s2
min_s2
delta_s2
max_s3
min_s3
delta_s3
max_s4
min_s4
delta_s4
max_s5
min_s5
delta_s5
max_s6
min_s6
delta_s6
max_s7
min_s7
delta_s7
max_s8
min_s8
delta_s8
max_s9
min_s9
delta_s9
max_s10
min_s10
delta_s10
max_s11
min_s11
delta_s11
max_s12
min_s12
delta_s12
max_s13
min_s13
delta_s13
max_s14
min_s14
delta_s14
max_s15
min_s15
delta_s15
max_s16
min_s16
delta_s16
max_s17
min_s17
delta_s17
max_s18
min_s18
delta_s18
max_s19
min_s19
delta_s19
max_s20
min_s20
delta_s20
max_s21
min_s21
delta_s21


In [39]:
random_engine = np.random.randint(1,100)
val_data = data.loc[data.engine_id == random_engine].iloc[:,2:].copy()    
RUL_df = data.loc[data.engine_id != random_engine].iloc[:,2:].copy()

In [44]:
labels = RUL_df['RUL']
features = RUL_df.iloc[:,1:]
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, random_state = 42)

In [45]:
gbr_non_opt = ensemble.GradientBoostingRegressor()
gbr_non_opt.fit(X_train, y_train)

GradientBoostingRegressor()

In [48]:
pred_non_opt = gbr_non_opt.predict(X_test)
print('MAE: %s' % metrics.mean_absolute_error(y_test, pred_non_opt))
print('MSE: %s' % metrics.mean_squared_error(y_test, pred_non_opt))

MAE: 9.200445326669886
MSE: 142.63671398695553


In [49]:
n_estimators = [150, 250, 300, 400]
max_depth = [5, 10, 50, 100, None]
max_features = ['auto', 'sqrt']
min_samples_split= [2, 3, 5, 10]
min_samples_leaf= [1, 2, 4]

random_grid = {'n_estimators' : n_estimators,
               'max_depth' : max_depth,
               'max_features' : max_features,
               'min_samples_split' : min_samples_split,
               'min_samples_leaf' : min_samples_leaf
}

In [ ]:
gbr = ensemble.GradientBoostingRegressor()
gbr_random = RandomizedSearchCV(estimator = gbr, param_distributions = random_grid, n_iter = 10, cv = 3, verbose = 2)
gbr_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_estimators=300, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=None 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=300, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=None, total= 1.1min
[CV] n_estimators=300, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=None 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  n_estimators=300, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=None, total= 1.1min
[CV] n_estimators=300, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=None 
[CV]  n_estimators=300, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=None, total= 1.1min
[CV] n_estimators=300, min_samples_split=3, min_samples_leaf=2, max_features=sqrt, max_depth=100 
[CV]  n_estimators=300, min_samples_split=3, min_samples_leaf=2, max_features=sqrt, max_depth=100, total= 1.1min
[CV] n_estimators=300, min_samples_split=3, min_samples_leaf=2, max_features=sqrt, max_depth=100 
[CV]  n_estimators=300, min_samples_split=3, min_samples_leaf=2, max_features=sqrt, max_depth=100, total= 1.3min
[CV] n_estimators=300, min_samples_split=3, min_samples_leaf=2, max_features=sqrt, max_depth=100 
[CV]  n_estimators=300, min_samples_split=3, min_samples_leaf=2, max_features=sqrt, max_depth=100, total= 1.2min
[CV] n_estimators=300, min_samples_split

In [ ]:
with open(best_param.json, 'w') as outfile:
    json.dump(gbr_random.best_params_, outfile)

In [ ]:
pprint(gbr_random.best_params_)

In [ ]:
X_val, y_val = val_data.iloc[:,1:], val_data.iloc[:,0]

val_pred_non_opt = gbr_non_opt.predict(X_val)
val_pred_random = gbr_random.predict(X_val)
print('MAE: %s' % metrics.mean_absolute_error(y_val val_pred_non_opt))
print('MSE: %s' % metrics.mean_squared_error(y_val, val_pred_non_opt))
print('MAE: %s' % metrics.mean_absolute_error(y_val, val_pred_random))
print('MSE: %s' % metrics.mean_squared_error(y_val, val_pred_random))

In [ ]:
if metrics.mean_absolute_error(y_val val_pred_non_opt)) < metrics.mean_absolute_error(y_val, val_pred_random)):
    gbr_best = gbr_random.best_estimator
    print('optimized version is best')
else:
    gbr_best = gbr_non_opt
    print('non optimized version is best')

In [ ]:
with open(tmp_dir + 'model.pickle', 'wb') as f:
    cPickle.dump(gbr_best, f, -1)

In [ ]:
client = storage.Client()
bucket = client.get_bucket('RUL_prediction')
blob = bucket.blob(model_path+'model.pickle')
blob.upload_from_filename('/tmp/model.pickle')